# D-Score Suite (v1) Benchmark -- Usage Examples

>_This notebook adapted from originals by Timothy Hodson and Rich Signell. See that upstream work at:_
>* https://github.com/thodson-usgs/dscore
>* https://github.com/USGS-python/hytest-evaluation-workflows/

This notebook will demonstrate how to call the specific functions defined in the d-score metrics suite notebook (Metrics_Dscore_Suite_v1.ipynb), using a small demonstration dataset.

In [ ]:
import pandas as pd
import numpy as np

## Sample Data

In [ ]:
sampleData = pd.read_csv(r"../streamflow/NWM_Benchmark_SampleData.csv", index_col='date', parse_dates=True).dropna()
print(len(sampleData.index), " Records")

A quick look at the table shows that this data contains time-series streamflow values for
observed ('obs'), the NWM data model ('nwm'), and the NHM model ('nhm').  This demonstration
dataset limits to a single gage ("`site_no` = 01104200")

In [ ]:
sampleData.head()

## Import Benchmark Functions
The metric functions are defined and described in
{doc}`/evaluation/Metrics_DScore_Suite_v1`.
They are imported here by running that notebook from within the following cell:

In [ ]:
%run ../../Metrics_DScore_Suite_v1.ipynb

The functions are now available here, to run against our sample data:

In [ ]:
# Mean Square Error
mse(sampleData['obs'], sampleData['nwm'])

In [ ]:
seasonal_mse(sampleData['obs'], sampleData['nwm'])

## Create Composite Benchmark
It is useful to combine several of these metrics into a single benchmark routine, which returns a pandas Series of the assembled metrics.

This 'wrapper' composite benchmark also handles any transforms of the data before calling the metric functions. In this case, we will log transform the data. 

In [ ]:
def compute_benchmark(df):
    """
    Runs several metrics against the data table in 'df'.  

    NOTE: the 'obs' and 'nwm' columns must exist in df, and that nan's have already been removed.  
    """
    obs = np.log(df['obs'].clip(lower=0.01)) # clip to remove zeros and negative values
    sim = np.log(df['nwm'].clip(lower=0.01)) 
    
    mse_ = pd.Series(
        [ mse(obs, sim) ], 
        index=["mse"], 
        dtype='float32'
    )
    return pd.concat([
            mse_,
            bias_distribution_sequence(obs, sim), 
            stl(obs, sim),
            seasonal_mse(obs, sim),
            quantile_mse(obs, sim)
            ],
        )

In [ ]:
compute_benchmark(sampleData)

## Score-Cards
The D-score functions include an ILAMB-style scorecard function to produce a graphic scorecard from these metrics.
> Note that a scorecard such as this is typically applied to a composite of D-score metrics computed for many gages.
> This demos the scorecard for a single gage **as if** it were the mean of all gages in an evaluation analysis. 

In [ ]:
# Compute benchmark and 'score' each decomp as percent of total MSE
bm = compute_benchmark(sampleData)
percentage_card = pd.DataFrame(data={
    'NWM' : ((bm / bm['mse']) * 100).round().astype(int)
    })
percentage_card.name="Percent"  ## NOTE: `name` is a non-standard attribute for a dataframe. We use it to stash
                                ## metadata for this dataframe which the ilamb_card_II() func will use to label things.
percentage_card

In [ ]:
n_cards=1
fig, ax = plt.subplots(1, n_cards, figsize=(0.5+(1.5*n_cards), 3.25), dpi=150)
ax = ilamb_card_II(percentage_card, ax)
plt.show()

In [ ]:
## if the score card has columns for multilple models.....  
# fictitious example:
percentage_card['XYZ'] = pd.Series([100, 20, 30, 20, 10, 50, 60, 70, 20, 10, 40, 65, 15,10,5], index=percentage_card.index)
fig, ax = plt.subplots(1, n_cards, figsize=(0.5+(1.5*n_cards), 3.25), dpi=150)
ax = ilamb_card_II(percentage_card, ax)
plt.show()